In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
df=pd.read_excel("D:\\DATASETS\\deepinsurance.xlsx")

In [5]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=56)

In [7]:
x_train.shape

(22, 2)

In [8]:
x_test.shape

(6, 2)

In [9]:
x_train_scalled=x_train.copy()
x_train_scalled['age']=x_train_scalled['age']/100

x_test_scalled=x_test.copy()
x_test_scalled['age']=x_test_scalled['age']/100

In [10]:
x_train_scalled.head()

,age,affordibility
18,0.19,0
8,0.62,1
1,0.25,0
6,0.55,0
5,0.56,1


In [11]:
x_test_scalled.head()

,age,affordibility
10,0.18,1
7,0.60,0
16,0.25,0
3,0.52,0
20,0.21,1


In [17]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')#kernel_initializer means weights
 ])
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_scalled,y_train,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.6218 - accuracy: 0.5455
Epoch 2/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6215 - accuracy: 0.5455
Epoch 3/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6212 - accuracy: 0.5455
Epoch 4/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.6210 - accuracy: 0.5455
Epoch 5/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6207 - accuracy: 0.5455
Epoch 6/1000
1/1 [==============================] - 0s 72ms/step - loss: 0.6204 - accuracy: 0.5455
Epoch 7/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6201 - accuracy: 0.5455
Epoch 8/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6198 - accuracy: 0.5455
Epoch 9/1000
1/1 [==============================] - 0s 80ms/step - loss: 0.6195 - accuracy: 0.5455
Epoch 10/1000
1/1 [==============================] - 0s 48ms/step - loss: 0.6193 - accuracy: 0.5455
Epoch 11/10

In [18]:
model.evaluate(x_test_scalled,y_test)

1/1 [==============================] - 1s 678ms/step - loss: 0.9972 - accuracy: 0.1667


[0.9971561431884766, 0.1666666716337204]

In [19]:
x_test_scalled

,age,affordibility
10,0.18,1
7,0.60,0
16,0.25,0
3,0.52,0
20,0.21,1
19,0.18,1


In [20]:
y_pred=model.predict(x_test_scalled)

1/1 [==============================] - 1s 895ms/step


In [21]:
y_pred

array([[0.6911772 ],
       [0.4140763 ],
       [0.3496389 ],
       [0.39899585],
       [0.69615835],
       [0.6911772 ]], dtype=float32)

In [22]:
y_test

10    0
7     1
16    1
3     0
20    0
19    0
Name: bought_insurance, dtype: int64

Calculating MSE

In [23]:
predicted=y_pred.reshape(-1)

In [24]:
def mse(true,y_pred):
    total_error=0
    for i,j in zip(true,y_pred):
        total_error+=abs(i-j)**2
    print("Total Error:",total_error)
    mean_squared_error=total_error/len(true)
    return "Mean_Squared_Error:",mean_squared_error


In [25]:
mse(y_test,predicted)

Total Error: 2.3655620798492825


('Mean_Squared_Error:', 0.3942603466415471)

In [26]:
coef,intercetpt=model.get_weights()

In [27]:
coef,intercetpt

(array([[0.78139263],
        [1.4809529 ]], dtype=float32),
 array([-0.81597495], dtype=float32))

Manualy making nural network

In [28]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [29]:
def predicted_function(w1,w2):
    weighted_sum=coef[0]*w1+coef[1]*w2+intercetpt
    return sigmoid(weighted_sum)

In [30]:
predicted_function(0.18,1)

0.6911771908042915

In [31]:
def log_loss(y_true,y_predicted):
    epsilon=1e-15
    y_predicted_new=[max(i,epsilon) for i in y_predicted]
    y_predicted_new=[min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new=np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [32]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [41]:
def gredient_descent(age,affordibility,y_true,epochs,loss_thresold):
    w1=w2=1
    bias=0
    learning_rate=0.5
    n=len(age)

    for i in range(epochs):
        weighted_sum=w1*age+w2*affordibility+bias
        y_predicted=sigmoid_numpy(weighted_sum)
        loss=log_loss(y_true,y_predicted)
        w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d=(1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
        bias_d=np.mean(y_predicted-y_true)

        w1=w1-learning_rate*w1d
        w2=w2-learning_rate*w2d
        bias=bias-learning_rate*bias_d

        print(f'Epochs:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
    
    return w1, w2, bias 

In [43]:
gredient_descent(x_train_scalled['age'],x_train_scalled['affordibility'],y_train,1000,0.5079)

Epochs:0, w1:0.9798444627299807, w2:0.9776719874706713, bias:-0.10067789979613982, loss:0.6218130862091308
Epochs:1, w1:0.9642612597356127, w2:0.9629838979398024, bias:-0.190144880473967, loss:0.6011086246399742
Epochs:2, w1:0.9528002091597373, w2:0.9551969782251923, bias:-0.26952885742426286, loss:0.5852421311355965
Epochs:3, w1:0.9449661346807738, w2:0.9534670538560527, bias:-0.3400433925264369, loss:0.5730427057947375
Epochs:4, w1:0.9402603583696084, w2:0.9569248959682491, bias:-0.40289366426478307, loss:0.5635408562701678
Epochs:5, w1:0.9382100708634803, w2:0.9647348042591565, bias:-0.45921161029548996, loss:0.5559791238776005
Epochs:6, w1:0.9383858641666188, w2:0.9761309456566455, bias:-0.5100185659467861, loss:0.5497916530547977
Epochs:7, w1:0.9404097800191026, w2:0.9904354024693143, bias:-0.5562094332680281, loss:0.5445699677153633
Epochs:8, w1:0.9439567446420566, w2:1.0070631921030506, bias:-0.5985515658028603, loss:0.5400267205491448
Epochs:9, w1:0.9487519287548781, w2:1.02551

(8.594774870459354, 4.105415721301576, -6.785637203923945)

In [44]:
coef, intercetpt

(array([[0.78139263],
        [1.4809529 ]], dtype=float32),
 array([-0.81597495], dtype=float32))